<a href="https://colab.research.google.com/github/jaihonikhil/Group-15/blob/master/Preprocessing_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREPROCESSING CODE FOR XML FILE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/AutogAnnotationsTest.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/TAPAS/FINALTEST')

In [3]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/ManAnnotationsTest.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/TAPAS/FINALTEST')

In [4]:
from os import walk
mypathTestIAA="/content/drive/MyDrive/TAPAS/FINALTEST/Autogenerated Annotations/Test/input"
mypathTestIMA="/content/drive/MyDrive/TAPAS/FINALTEST/Manual Annotations/Test/input"
mypathDevI="/content/drive/My Drive/Manual Annotations/Dev/input"
mypathDevO="/content/drive/My Drive/Manual Annotations/Train/output"
# mypathDevO="/content/drive/My Drive/Manual Annotations/Dev/output"

In [13]:
import os
files_in_directory = os.listdir(mypathTestIMA)#EDIT THIS FOR ANOTHER DIRECTORY
examples = [file for file in files_in_directory if file.endswith(".xml")]
for e in range(len(examples)):
  examples[e]=examples[e][:-4]


In [14]:
def clean_cell(content):
  content=content.replace("n/a","")
  bad_words = ['\n','#',',','/','"','$']
  for word in bad_words : 
    content=content.replace(word," ")
  content=content.replace("++","increases")
  content=content.replace("--","decreases")
  content=content.replace("+","positive")
  content=content.replace("-","negative")


  return content

In [15]:
len(examples)

42

In [16]:
#EDIT THIS FOR TABLE MAKIING IN CSV TASK 1 
from bs4 import BeautifulSoup as bs
import numpy as np
import json
import csv 
import re
import pandas as pd
filename=[]
stateme=[]
ind=[]
captio=[]
jj=0
for example1 in examples:
  content = []
  with open(mypathTestIMA+"/"+example1+".xml", "r") as file:#EDIT THIS FOR ANOTHER DIRECTORY
      content = file.readlines()
      content = "".join(content)
      soup = bs(content, "lxml")
  matrices=[]
  tables = soup.find_all('table')
  num_tables = len(tables)
  p=0

  for t in tables:
    p=p+1  
    row=t.find_all('row')
    capt=t.find('caption')
    legendtxt=t.find('legend')
    if legendtxt != None:
      # print(capt['text'])
      capt['text']=capt['text']+" "+legendtxt['text']
      # print("HI")
      # print(example1+"."+str(p)+"DMO.html.csv")
      # print(capt['text'])
    # print(capt)
    statements=t.find_all('statement')
    state=[]
    index=[]
    for s in statements:
      a_list = s['text'].split()
      s['text'] = " ".join(a_list)
      s['text']=str(s['text'])
      s['text']=s['text'].replace("#","")
      # print(s['text'])
      if (s['type']=='entailed'):
        state.append(s['text'])
        s['type']=1
        index.append(s['type'])
      elif (s['type']=='refuted'):
        state.append(s['text'])
        s['type']=0
        index.append(s['type'])
      else:
        state.append(s['text'])
        s['type']=1
        index.append(s['type'])
        jj=jj+1
    
    stateme.append(state)
    ind.append(index)
    if (capt!=None):
      a_list = capt['text'].split()
      capt['text'] = " ".join(a_list)
      capt['text'] = re.sub(' +', ' ',capt['text']) 
      capt['text']=capt['text'].replace("#","")
      # print(capt['text'])
      captio.append(str(capt['text']))
    else:
      captio.append(" ")
    ####################################################################################################################################################
    cells=[]
    for s in row:
      table_s = s.find_all('cell') 
      cells.append(len(table_s))
    m=max(cells)# For intrapolating hence finding maximum number of columns
    matrix=[]
    lex=[]

    array=[]
    for l in range(len(row)):
      array.append("")
    # array=" "
    for r in row:
      all_cells = r.find_all('cell')
      j=0
      # x=
      s=""
      rowstring=""
      for j in all_cells:
        cend=j['col-end']
        cstart=j['col-start']
        rend=int(j['row-end'])
        rstart=int(j['row-start'])
        j['text']=j['text'].replace('#', '')
        if (cend==cstart):
          array[rstart]=array[rstart]+j['text']+'#'
        else:
          kk=int(cend)-int(cstart)+1
          for b in range(kk):
            array[rstart]=array[rstart]+j['text']+'#'
        if (rend!=rstart):
          kk=int(rend)-int(rstart)
          for b in range(kk):
            array[rstart+b+1]=array[rstart+b+1]+j['text']+'#'
        
        a_list = array[rstart].split()
        
        array[rstart] = " ".join(a_list)
        
    matrix=[]
    for l in array:
      l=l[:-1]
      a_list = l.split("#")
      matrix.append(a_list)
      # print(len(a_list))
    h = matrix[0]
    # print(h)
    matrix =matrix[1:]
    filename.append(example1+"."+str(p)+"TESTMAN.html.csv")

    df = pd.DataFrame.from_records(matrix, columns=h)
   
    df.to_csv("/content/drive/My Drive/TAPAS/data/TESTcsv/"+ example1  +"." + str(p) + "TESTMAN" + '.html.csv',sep='#',mode = 'w', index=False, index_label=False)
    #df.to_csv("/content/drive/My Drive/Updated_dataset/all_auto/"+ example1  +"." + str(p) +  "DAO" + '.html.csv',sep='#',mode = 'w', index=False, index_label=False)
    #df.to_csv("/content/drive/My Drive/Updated_dataset/DAO/"+ example1  +"." + str(p) + "DAO" + '.html.csv',sep='#',mode = 'w', index=False, index_label=False)
print(jj)
print("cool")

456
cool


In [ ]:
16839+ 456

In [17]:
import json 
 
json_object = json.dumps(filename, indent = 2) 
  
# Writing to sample.json 
with open("/content/drive/My Drive/TAPAS/REALTestOutputM_IDS.json", "w") as outfile: 
    outfile.write(json_object) 

In [18]:
len(filename)

98

In [19]:
#Creates File For Json Writing 
feeds={}
with open("/content/drive/My Drive/TAPAS/REALTestOutputM.json", "w",encoding='utf8') as write_file:#EDIT THIS FOR ANOTHER DIRECTORY
  for i in range(len(filename)):
    a_list = captio[i].split()
    new_string = " ".join(a_list)
    entry ={filename[i]:[stateme[i], ind[i],new_string]}
    feeds.update(entry)
    i=i+1
  json.dump(feeds,write_file,indent=2,ensure_ascii=False, )
    # json.dumps(feeds)
 
#Json writing ends

**Autogenerated**

> Train Output-1591 Tables 695 files

> Dev Output-195 Tables 86 files

**Manual**


> Train Output- 783  Tables 340 files


> Dev Output-100 Tables 42 files

In [ ]:
# CODE THIS FOR TASK 2
from bs4 import BeautifulSoup as bs
import numpy as np
import json
import csv 
import re
filename=[]
stateme=[]
ind=[]
captio=[]
# for example1 in examples:
content = []
matt=[]

for example1 in examples:
  content = []
with open('/content/20198.xml', "r") as file:#EDIT THIS FOR ANOTHER DIRECTORY
    content = file.readlines()
    content = "".join(content)
    soup = bs(content, "lxml")
matrices=[]
tables = soup.find_all('table')
num_tables = len(tables)
p=0

for t in tables:
  p=p+1  
  row=t.find_all('row')
  capt=t.find('caption')
  statements=t.find_all('statement')
  state=[]
  index=[]
  cells=[]
  for s in row:
    table_s = s.find_all('cell')
    cells.append(len(table_s))
  m=max(cells)# For intrapolating hence finding maximum number of columns
  mat = np.zeros((len(statements),len(row),m))
  matrix=[]
  lex=[]
  for r in row:
    table_rows = r.find_all('cell')
    for c in table_rows:
      ce=c.find_all('evidence')
      cs=int(c['col-start'])
      cee=int(c['col-end'])
      rs=int(c['row-start'])
      re=int(c['row-end'])
      for j in ce:
        f=int(j['statement_id'])
        k=j.find('type')
        if (j['type']=='relevant'):
          for c in range(rs,re+1):
            for k in range(cs,cee+1):
              mat[f][c][k]=1
  matt.append(mat)      




In [ ]:
import os
files_in_directory = os.listdir("/content/drive/My Drive/Group-15-Git/Table-Fact-CheckingModified/all_programs")#EDIT THIS FOR ANOTHER DIRECTORY
examples = [file for file in files_in_directory]
examples

In [ ]:
#old interpolated code 

from bs4 import BeautifulSoup as bs
import numpy as np
import json
import csv 
import pandas as pd
filename=[]
stateme=[]
ind=[]
captio=[]
# for example1 in examplescontent = []
  # with open(B+"/"+example1+".xml", "r") as file:#EDIT THIS FOR ANOTHER DIRECTORY
with open("/content/20189.xml", "r") as file:
    content = file.readlines()
    content = "".join(content)
    soup = bs(content, "lxml")
matrices=[]
tables = soup.find_all('table')
num_tables = len(tables)
p=0

for t in tables:
  p=p+1  
  row=t.find_all('row')
  #print(row)
  capt=t.find('caption')
  statements=t.find_all('statement')
  state=[]
  index=[]
  for s in statements:
    a_list = s['text'].split()
    s['text'] = " ".join(a_list)
    s['text']=str(s['text'])
    # print(s['text'])
    if (s['type']=='entailed'):
      state.append(s['text'])
      s['type']='1'
      index.append(s['type'])
    elif (s['type']=='refuted'):
      state.append(s['text'])
      s['type']='0'
      index.append(s['type'])
  
  stateme.append(state)
  ind.append(index)
  if (capt!=None):
    a_list = capt['text'].split()
    capt['text'] = " ".join(a_list)
    captio.append(capt['text'])
  else:
    captio.append(" ")
  cells=[]
  for s in row:
    table_s = s.find_all('cell')
    cells.append(len(table_s))
  m=max(cells)# For intrapolating hence finding maximum number of columns
  matrix=[]
  for r in row:
    table_rows = r.find_all('cell')
    #print(table_rows)
    j=0
    x=0
    s=""
    rowstring=""
    for j in range(m):
      l=table_rows[x]
      if(j==int(l['col'])):
        cell_content = clean_cell(l['text'])
        #print(cell_content)
        rowstring = rowstring + cell_content + '#'
        # print(rowstring)
        s=l['text']
        if(x<len(table_rows)-1):
          x=x+1
      else :
        # print("#######################")
        #print(s)
        rowstring=rowstring+s+'#'
      j=j+1
    rowstring=rowstring[:-1]
    a_list = rowstring.split()
    new_string = " ".join(a_list)
    #print(rowstring)
    a_list = new_string.split("#")
    #print(a_list)
    matrix.append(a_list)
  
  h = matrix[0]
  # print(matrix)
  #print(len(h))
  matrix = matrix[1:]
  #for columns in matrix: 
    #print(len(columns))
    #print(columns)
  #print(len(matrix[0]))
  print(matrix)
  df = pd.DataFrame.from_records(matrix, columns=h)
  # print(df)
  # df.to_csv("/content/drive/My Drive/TAPAS/all_csv/"+ example1  +"." + str(p) + "DMO" + '.html.csv',sep='#',mode = 'w', index=False, index_label=False)
  #df.to_csv("/content/drive/My Drive/Updated_dataset/all_auto/"+ example1  +"." + str(p) +  "DAO" + '.html.csv',sep='#',mode = 'w', index=False, index_label=False)
  #df.to_csv("/content/drive/My Drive/Updated_dataset/DAO/"+ example1  +"." + str(p) + "DAO" + '.html.csv',sep='#',mode = 'w', index=False, index_label=False)


[['', ' Min ', ' Max ', ' Min ', ' Max ', ' Min ', ' Max'], ['Desktop ', ' 24 × 31 ', ' 37.5 × 37.5 ', ' 6 ', ' 165 ', ' 80 ', ' 105'], ['Mobile ', ' 12 × 12 ', ' 42 × 24 ', ' 2.2 ', ' 57 ', ' 80 ', ' 105'], ['Server ', ' 34 × 28 ', ' 88 × 56.5 ', ' 11.7 ', ' 320 ', ' 40 ', ' 100'], ['Embedded ', ' 25 × 27 ', ' 45 × 42.5 ', ' 3 ', ' 105 ', ' 40 ', ' 110']]
[['Ponegative210 ', ' Metal ', ' 1210 ', ' 138 d ', ' 254 ', ' 9.3 ', ' 31.7 ', ' Alpha'], ['Pmnegative147 ', ' Pm 2 O 3 ', ' 1.5 ', ' 2.6 y ', ' 2300 ', ' 5.55 ', ' 0.37 ', ' Beta'], ['Srnegative90 ', ' SrO 2 ', ' 0.93 ', ' 28 y ', ' 2430 ', ' 2.65 ', ' 6.5 ', ' Beta'], ['Punegative238 ', ' PuO 2 ', ' 5.0 ', ' 89.6 y ', ' 2250 ', ' 11.46 ', ' 34.5 ', ' Alpha'], ['Cenegative144 ', ' CeO 2 ', ' 13.8 ', ' 285 d ', ' 2680 ', ' 7 ', ' 7.9 ', ' Beta']]


In [ ]:

#OLD INTRAPOLATION CODE 

from bs4 import BeautifulSoup as bs
import numpy as np
import json
import csv 
import re
filename=[]
stateme=[]
ind=[]
captio=[]
for example1 in examples:
  content = []
  with open(mypathDevO+"/"+example1+".xml", "r") as file:#EDIT THIS FOR ANOTHER DIRECTORY
      content = file.readlines()
      content = "".join(content)
      soup = bs(content, "lxml")
  matrices=[]
  tables = soup.find_all('table')
  num_tables = len(tables)
  p=0
  
  for t in tables:
    p=p+1  
    row=t.find_all('row')
    capt=t.find('caption')
    legendtxt=t.find('legend')
    if legendtxt != None:
      # print(capt['text'])
      capt['text']=capt['text']+legendtxt['text']
      # print("HI")
      # print(capt['text'])
    # print(capt)
    statements=t.find_all('statement')
    state=[]
    index=[]
    for s in statements:
      a_list = s['text'].split()
      s['text'] = " ".join(a_list)
      s['text']=str(s['text'])
      s['text']=s['text'].replace("#","")
      # print(s['text'])
      if (s['type']=='entailed'):
        state.append(s['text'])
        s['type']=1
        index.append(s['type'])
      elif (s['type']=='refuted'):
        state.append(s['text'])
        s['type']=0
        index.append(s['type'])
    
    stateme.append(state)
    ind.append(index)
    if (capt!=None):
      a_list = capt['text'].split()
      capt['text'] = " ".join(a_list)
      capt['text'] = re.sub(' +', ' ',capt['text']) 
      capt['text']=capt['text'].replace("#","")
      # print(capt['text'])
      captio.append(str(capt['text']))
    else:
      captio.append(" ")
    cells=[]
    for s in row:
      table_s = s.find_all('cell')
      cells.append(len(table_s))
    m=max(cells)# For intrapolating hence finding maximum number of columns
    matrix=[]
    lex=[]
    for r in row:
      table_rows = r.find_all('cell')
      j=0
      x=0
      s=""
      rowstring=""
      for j in range(m):
        l=table_rows[x]
        if(j==int(l['col'])):
          l['text']=l['text'].replace("#","")
          rowstring=rowstring+l['text']+'#'
          s=l['text']
          if(x<len(table_rows)-1):
            x=x+1
        else :
          rowstring=rowstring+s+'#'
        j=j+1
      rowstring=rowstring[:-1]
      a_list = rowstring.split()
      new_string = " ".join(a_list)
      lex.append(new_string)
      matrix.append(lex)
      lex=[]
      s=""
      rowstring=""
    filename.append(example1+"."+str(p)+"DAO.html.csv")
    dir="/content/drive/My Drive/DeleteOutput/"#EDIT THIS FOR ANOTHER DIRECTORY
    file = open(dir+example1+"."+str(p)+'DAO.html.csv', 'w', newline ='') 
    with file:     
      write = csv.writer(file) 
      write.writerows(matrix) 
    matrices.append(matrix)
